In [1]:
import pandas as pd
import polars as pl
import dask.dataframe as dd
import dask.multiprocessing
import xarray as xr

In [2]:
bathy = xr.open_dataset("/home/an1n23/datasets/IBCSO_v2_ice-surface_WGS84.nc")

In [7]:
def get_depth(row):
    return bathy.z.interp(lat=row['LATITUDE'], lon=row['LONGITUDE']).item()
def get_depth_nearest(row):
    return bathy.z.sel(lat=row['LATITUDE'], lon=row['LONGITUDE'], method="nearest").item()
    
def assign_echodepth(df):
    ind = df.index
    lon = df.loc[ind[0], 'LONGITUDE']
    lat = df.loc[ind[0],'LATITUDE']
    ind_lon = np.argmin(np.abs(lons - lon))
    ind_lat = np.argmin(np.abs(lats - lat))
    df.loc[:,'ECHODEPTH'] = bathy.variables['elevation'][ind_lat, ind_lon]

    return df

In [4]:
!ls /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/*.part > arbit_filelist.txt

In [5]:
meopfiles = pd.read_csv("arbit_filelist.txt", header=None)

In [6]:
!rm /scratch/an1n23/meop_RS_AS/*.part

In [49]:
!mkdir /scratch/an1n23/meop_RS_AS

In [13]:
meopfiles[0][0].split("/")[-1]

'00.part'

In [14]:
for f in meopfiles[0][:]:
    print("Processing file:", f)
    
    meop = pd.read_csv(f)
    unnamed_cols = [col for col in meop.columns if col.startswith("Unnamed")]
    meop = meop.drop(columns = unnamed_cols)
    sel_lat = meop["LATITUDE"] <= -50
    
    meop = meop.loc[sel_lat]


    unique_coords = meop.drop_duplicates(subset=['LATITUDE', 'LONGITUDE'])\
                                       [['LATITUDE', 'LONGITUDE']]
    unique_coords["ECHODEPTH"] = unique_coords.apply(get_depth_nearest, axis=1)
    
    meop = meop.merge(unique_coords, on=['LATITUDE', 'LONGITUDE'], how='left')
    
    meop.to_csv("/scratch/an1n23/meop_RS_AS/"+f.split("/")[-1], index=False)
    
    del(meop)
        
        
    

Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/00.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/01.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/02.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/03.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/04.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/05.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/06.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/07.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/08.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/09.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/10.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/11.part
Processing file: /home/an1n23/datasets/merged_RS_AS_MEOP26112021.csv/12.part

In [ ]:
!echo "completed meop assign bathy 30092023" > arbit.txt